# Library

In [1]:
import numpy as np 
import pandas as pd 
import os
from IPython.display import Markdown
from datetime import timedelta
import json, requests
from datetime import datetime
import glob
import requests 
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import folium
%matplotlib inline
import seaborn as sns
sns.set()
import pycountry
from plotly.offline import init_notebook_mode, iplot 
import plotly.offline as py
import plotly.express as ex
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
py.init_notebook_mode(connected=True)
import folium 
from folium import plugins
plt.style.use("seaborn-talk")
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['image.cmap'] = 'viridis'
from fbprophet import Prophet
pd.set_option('display.max_rows', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/covid19-india-cleaned-data/complete.csv
/kaggle/input/coronavirus-cases-in-india/per_day_cases.xlsx
/kaggle/input/coronavirus-cases-in-india/Indian Coordinates.csv
/kaggle/input/coronavirus-cases-in-india/Covid cases in India.csv
/kaggle/input/covid19-corona-virus-india-dataset/complete.csv
/kaggle/input/covid19-corona-virus-india-dataset/web_scraping.ipynb
/kaggle/input/covid19-corona-virus-india-dataset/patients_data.csv
/kaggle/input/covid19-corona-virus-india-dataset/patient_wise_data_scrapping_and_cleaning.ipynb
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths_US.csv
/kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv
/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed_US.csv

# Uploading Data

In [2]:
link = 'https://www.mohfw.gov.in/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")
thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[-1]
body = tbody.find_all('tr')
head_rows = []
body_rows = []
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)   
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
df_bs = pd.DataFrame(body_rows[:len(body_rows)-1], columns=head_rows[0])   
df_bs.drop('S. No.', axis=1, inplace=True)
#To remove last raw
df_bs.drop(df_bs.tail(1).index,axis = 0,inplace=True)
now  = datetime.now()
df_bs['Date'] = now.strftime("%m/%d/%Y") 
df_bs['Date'] = pd.to_datetime(df_bs['Date'], format='%m/%d/%Y')

In [3]:
locations = {
    "Kerala" : [10.8505,76.2711],
    "Maharashtra" : [19.7515,75.7139],
    "Karnataka": [15.3173,75.7139],
    "Telangana": [18.1124,79.0193],
    "Uttar Pradesh": [26.8467,80.9462],
    "Rajasthan": [27.0238,74.2179],
    "Gujarat":[22.2587,71.1924],
    "Delhi" : [28.7041,77.1025],
    "Punjab":[31.1471,75.3412],
    "Tamil Nadu": [11.1271,78.6569],
    "Haryana": [29.0588,76.0856],
    "Madhya Pradesh":[22.9734,78.6569],
    "Jammu and Kashmir":[33.7782,76.5762],
    "Ladakh": [34.1526,77.5770],
    "Andhra Pradesh":[15.9129,79.7400],
    "West Bengal": [22.9868,87.8550],
    "Bihar": [25.0961,85.3131],
    "Chhattisgarh":[21.2787,81.8661],
    "Chandigarh":[30.7333,76.7794],
    "Uttarakhand":[30.0668,79.0193],
    "Himachal Pradesh":[31.1048,77.1734],
    "Goa": [15.2993,74.1240],
    "Odisha":[20.9517,85.0985],
    "Andaman and Nicobar Islands": [11.7401,92.6586],
    "Puducherry":[11.9416,79.8083],
    "Manipur":[24.6637,93.9063],
    "Mizoram":[23.1645,92.9376],
    "Assam":[26.2006,92.9376],
    "Meghalaya":[25.4670,91.3662],
    "Tripura":[23.9408,91.9882],
    "Arunachal Pradesh":[28.2180,94.7278],
    "Jharkhand" : [23.6102,85.2799],
    "Nagaland": [26.1584,94.5624],
    "Sikkim": [27.5330,88.5122],
    "Dadra and Nagar Haveli":[20.1809,73.0169],
    "Lakshadweep":[10.5667,72.6417],
    "Daman and Diu":[20.4283,72.8397]   
}

In [4]:
lat = {'Delhi':28.7041,
       'Haryana':29.0588,
       'Kerala':10.8505,
       'Rajasthan':27.0238,
       'Telengana':18.1124,
       'Uttar Pradesh':26.8467,
       'Ladakh':34.2996,
       'Tamil Nadu':11.1271,
       'Jammu and Kashmir':33.7782,
       'Punjab':31.1471,
       'Karnataka':15.3173,
       'Maharashtra':19.7515,
       'Andhra Pradesh':15.9129, 
       'Odisha':20.9517, 
       'Uttarakhand':30.0668, 
       'West Bengal':22.9868, 
       'Puducherry': 11.9416, 
       'Chandigarh': 30.7333, 
       'Chhattisgarh':21.2787, 
       'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 
       'Madhya Pradesh': 22.9734, 
       'Bihar': 25.0961, 
       'Manipur':24.6637,
       'Mizoram':23.1645,
        'Goa':15.2993,
     'Andaman and Nicobar Islands':11.7401}

long = {'Delhi':77.1025,
        'Haryana':76.0856,
        'Kerala':76.2711,
        'Rajasthan':74.2179,
        'Telengana':79.0193,
        'Uttar Pradesh':80.9462,
        'Ladakh':78.2932,
        'Tamil Nadu':78.6569,
        'Jammu and Kashmir':76.5762,
        'Punjab':75.3412,
        'Karnataka':75.7139,
        'Maharashtra':75.7139,
        'Andhra Pradesh':79.7400, 
        'Odisha':85.0985, 
        'Uttarakhand':79.0193, 
        'West Bengal':87.8550, 
        'Puducherry': 79.8083, 
        'Chandigarh': 76.7794, 
        'Chhattisgarh':81.8661, 
        'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 
        'Madhya Pradesh': 78.6569, 
        'Bihar': 85.3131, 
        'Manipur':93.9063,
        'Mizoram':92.9376,
         'Goa':74.1240,
       'Andaman and Nicobar Islands':92.6586}
df_bs['Latitude'] = df_bs['Name of State / UT'].map(lat)
df_bs['Longitude'] = df_bs['Name of State / UT'].map(long)
df_bs.head()

,Name of State / UT,Total Confirmed cases (Including 49 foreign Nationals),Cured/Discharged/Migrated,Death,Date,Latitude,Longitude
0,Andhra Pradesh,83,1,0,2020-04-01,15.9129,79.7400
1,Andaman and Nicobar Islands,10,0,0,2020-04-01,11.7401,92.6586
2,Bihar,23,0,1,2020-04-01,25.0961,85.3131
3,Chandigarh,13,0,0,2020-04-01,30.7333,76.7794
4,Chhattisgarh,9,0,0,2020-04-01,21.2787,81.8661


In [5]:
# complete data

file_name = now.strftime("%Y_%m_%d")+'.csv'
file_loc = ''
df_bs.to_csv(file_loc + file_name, index=False)
loc = ""
files = glob.glob(loc+'2020*.csv')
dfs = []
for i in files:
    df_temp = pd.read_csv(i)
    df_temp = df_temp.rename(columns={'Cured':'Cured/Discharged'})
    df_temp = df_temp.rename(columns={'Cured/Discharged':'Cured/Discharged/Migrated'})
    dfs.append(df_temp)
    
# print(dfs)

complete_data = pd.concat(dfs, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)
complete_data['Date'] = pd.to_datetime(complete_data['Date'])
complete_data = complete_data.sort_values(['Date', 'Name of State / UT']).reset_index(drop=True)

cols = ['Total Confirmed cases (Including 49 foreign Nationals) ','Cured/Discharged/Migrated', 'Death']
tot = complete_data.iloc[:,1]

complete_data[cols] = complete_data[cols].fillna(0).astype('int')

In [6]:
#old
'''
file_name = now.strftime("%Y_%m_%d")+'.csv'
file_loc = ''
df_bs.to_csv(file_loc + file_name, index=False)
loc = ""
files = glob.glob(loc+'2020*.csv')
dfs = []
for i in files:
    df_temp = pd.read_csv(i)
    df_temp = df_temp.rename(columns={'Cured':'Cured/Discharged'})
    df_temp = df_temp.rename(columns={'Cured/Discharged':'Cured/Discharged/Migrated'})
    dfs.append(df_temp)
complete_data = pd.concat(dfs, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)
complete_data['Date'] = pd.to_datetime(complete_data['Date'])
complete_data = complete_data.sort_values(['Date', 'Name of State / UT']).reset_index(drop=True)
cols = ['Total Confirmed cases (Indian National)', 'Total Confirmed cases ( Foreign National )', 
              'Cured/Discharged/Migrated', 'Death']
complete_data.to_csv('complete.csv', index=False'

'''

'\nfile_name = now.strftime("%Y_%m_%d")+\'.csv\'\nfile_loc = \'\'\ndf_bs.to_csv(file_loc + file_name, index=False)\nloc = ""\nfiles = glob.glob(loc+\'2020*.csv\')\ndfs = []\nfor i in files:\n    df_temp = pd.read_csv(i)\n    df_temp = df_temp.rename(columns={\'Cured\':\'Cured/Discharged\'})\n    df_temp = df_temp.rename(columns={\'Cured/Discharged\':\'Cured/Discharged/Migrated\'})\n    dfs.append(df_temp)\ncomplete_data = pd.concat(dfs, ignore_index=True).sort_values([\'Date\'], ascending=True).reset_index(drop=True)\ncomplete_data[\'Date\'] = pd.to_datetime(complete_data[\'Date\'])\ncomplete_data = complete_data.sort_values([\'Date\', \'Name of State / UT\']).reset_index(drop=True)\ncols = [\'Total Confirmed cases (Indian National)\', \'Total Confirmed cases ( Foreign National )\', \n              \'Cured/Discharged/Migrated\', \'Death\']\ncomplete_data.to_csv(\'complete.csv\', index=False\'\n\n'

In [7]:
data = pd.DataFrame(complete_data)
covid_19_India = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
df = pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv', parse_dates=['Date'])
india_data_json = requests.get('https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise').json()
df_india = pd.io.json.json_normalize(india_data_json['data']['statewise'])
df_india = df_india.set_index("state")

# Data Cleaning

In [8]:
df['Name of State / UT'] = df['Name of State / UT'].str.replace('Union Territory of ', '')

In [9]:
abc = pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv')
abc.rename(columns={'Name of State / UT':'State/UnionTerritory', 
                    'Total Confirmed cases (Indian National)':'ConfirmedIndianNational', 
                    'Total Confirmed cases ( Foreign National )':'ConfirmedForeignNational',
                    'Cured/Discharged/Migrated':'Cured',
                    'Death':'Deaths',
                    'Total Confirmed cases *' : 'Confirmed'                   
                   }, inplace=True)
abc.drop(['Latitude','Longitude'], axis=1, inplace=True)

In [10]:
#Later on use covid_19_India, due to missing data not using covid_19_India dataset
#covid_19_India['Confirmed'] = covid_19_India['ConfirmedIndianNational']+covid_19_India['ConfirmedForeignNational']
covid_19_India['Active']= covid_19_India['Confirmed']- (covid_19_India['Cured'] + covid_19_India['Deaths'])

In [11]:
abc['Confirmed'] = abc['ConfirmedIndianNational']+abc['ConfirmedForeignNational']
abc['Active']= abc['Confirmed']- (abc['Cured'] + abc['Deaths'])

In [12]:
data['Total cases'] = data['Total Confirmed cases (Including 49 foreign Nationals) ']
data['Active cases'] = data['Total cases'] - (data['Cured/Discharged/Migrated'] + data['Death'])

In [13]:
df = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Total Confirmed cases *', 'Death', 'Cured/Discharged/Migrated']]
df.columns = ['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Deaths', 'Cured']

for i in ['Confirmed', 'Deaths', 'Cured']:
    df[i] = df[i].astype('int')
    
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Cured']
df['Mortality rate'] = df['Deaths']/df['Confirmed']
df['Recovery rate'] = df['Cured']/df['Confirmed']

df = df[['Date', 'State/UT', 'Latitude', 'Longitude', 'Confirmed', 'Active', 'Deaths', 'Mortality rate', 'Cured', 'Recovery rate']]

In [14]:
latest = df[df['Date']==max(df['Date'])]

# days
latest_day = max(df['Date'])
day_before = latest_day - timedelta(days = 1)

# state and total cases 
latest_day_df = df[df['Date']==latest_day].set_index('State/UT')
day_before_df = df[df['Date']==day_before].set_index('State/UT')

temp = pd.merge(left = latest_day_df, right = day_before_df, on='State/UT', suffixes=('_lat', '_bfr'), how='outer')
latest_day_df['New cases'] = temp['Confirmed_lat'] - temp['Confirmed_bfr']
latest = latest_day_df.reset_index()
latest.fillna(1, inplace=True)

In [15]:
date_wise_data = df[['State/UT',"Date","Confirmed","Deaths","Cured",'Active']]
date_wise_data['Date'] = date_wise_data['Date'].apply(pd.to_datetime, dayfirst=True)
date_wise_data = date_wise_data.groupby(["Date"]).sum().reset_index()
def formatted_text(string):
    display(Markdown(string))
date_wise_data.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Confirmed,Deaths,Cured,Active
0,2020-01-30,1,0,0,1
1,2020-01-31,1,0,0,1
2,2020-02-01,2,0,0,2
3,2020-02-02,3,0,0,3
4,2020-02-03,3,0,0,3


In [16]:
#old 
'''
date_wise_data = covid_19_India[['State/UnionTerritory',"Date","Confirmed","Deaths","Cured",'Active']]
date_wise_data['Date'] = date_wise_data['Date'].apply(pd.to_datetime, dayfirst=True)
date_wise_data = date_wise_data.groupby(["Date"]).sum().reset_index()
def formatted_text(string):
    display(Markdown(string))
date_wise_data.head()
'''

'\ndate_wise_data = covid_19_India[[\'State/UnionTerritory\',"Date","Confirmed","Deaths","Cured",\'Active\']]\ndate_wise_data[\'Date\'] = date_wise_data[\'Date\'].apply(pd.to_datetime, dayfirst=True)\ndate_wise_data = date_wise_data.groupby(["Date"]).sum().reset_index()\ndef formatted_text(string):\n    display(Markdown(string))\ndate_wise_data.head()\n'

In [17]:
#
'''
df['Name of State / UT'] = df['Name of State / UT'].str.replace('Union Territory of ', '')
for i in ['Total Confirmed cases (Indian National)', 'Total Confirmed cases ( Foreign National )', 'Cured/Discharged/Migrated', 'Death']:
    df[i] = df[i].astype('int')    
#df['Total cases'] = df['Total Confirmed cases (Indian National)'] + df['Total Confirmed cases ( Foreign National )']
df.rename(columns={
                    'Total Confirmed cases *' : 'Total cases'                   
                   }, inplace=True)
df['Active'] = df['Total cases'] - df['Death'] - df['Cured/Discharged/Migrated']
df['Moratlity rate'] = df['Death']/df['Total cases']
df['Recovery rate'] = df['Cured/Discharged/Migrated']/df['Total cases']
#df.drop('Total Confirmed cases *', axis=1, inplace=True)
df.columns = ['Date', 'Name of State / UT', 'Confirmed(Indian)', 'Confirmed(Foreigner)', 'Cured/Discharged/Migrated', 'Latitude', 'Longitude', 'Death', 'Total cases', 'Mortality rate', 'Recovery rate', 'Active']
df = df[['Date', 'Name of State / UT', 'Latitude', 'Longitude', 'Confirmed(Indian)', 'Confirmed(Foreigner)', 'Total cases', 'Active', 'Death', 'Cured/Discharged/Migrated', 'Mortality rate', 'Recovery rate']'

'''

"\ndf['Name of State / UT'] = df['Name of State / UT'].str.replace('Union Territory of ', '')\nfor i in ['Total Confirmed cases (Indian National)', 'Total Confirmed cases ( Foreign National )', 'Cured/Discharged/Migrated', 'Death']:\n    df[i] = df[i].astype('int')    \n#df['Total cases'] = df['Total Confirmed cases (Indian National)'] + df['Total Confirmed cases ( Foreign National )']\ndf.rename(columns={\n                    'Total Confirmed cases *' : 'Total cases'                   \n                   }, inplace=True)\ndf['Active'] = df['Total cases'] - df['Death'] - df['Cured/Discharged/Migrated']\ndf['Moratlity rate'] = df['Death']/df['Total cases']\ndf['Recovery rate'] = df['Cured/Discharged/Migrated']/df['Total cases']\n#df.drop('Total Confirmed cases *', axis=1, inplace=True)\ndf.columns = ['Date', 'Name of State / UT', 'Confirmed(Indian)', 'Confirmed(Foreigner)', 'Cured/Discharged/Migrated', 'Latitude', 'Longitude', 'Death', 'Total cases', 'Mortality rate', 'Recovery rate', 

In [18]:
'''
df['Active'] = df['Total cases'] - df['Death'] - df['Cured/Discharged/Migrated']
'''

"\ndf['Active'] = df['Total cases'] - df['Death'] - df['Cured/Discharged/Migrated']\n"

In [19]:
'''
df.rename(columns={
                    'Name of State / UT' : 'State / UT'                   
                   }, inplace=True'
'''

"\ndf.rename(columns={\n                    'Name of State / UT' : 'State / UT'                   \n                   }, inplace=True'\n"

In [20]:
#old

'''
latest = df[df['Date']==max(df['Date'])]
latest_day = max(df['Date'])
day_before = latest_day - timedelta(days = 1)
latest_day_df = df[df['Date']==latest_day].set_index('State/UT')
day_before_df = df[df['Date']==day_before].set_index('State/UT')
temp = pd.merge(left = latest_day_df, right = day_before_df, on='State/UT', suffixes=('_lat', '_bfr'), how='outer')
latest_day_df['New cases'] = temp['Total cases_lat'] - temp['Total cases_bfr']
latest = latest_day_df.reset_index()
latest.fillna(1, inplace=True)

'''

"\nlatest = df[df['Date']==max(df['Date'])]\nlatest_day = max(df['Date'])\nday_before = latest_day - timedelta(days = 1)\nlatest_day_df = df[df['Date']==latest_day].set_index('State/UT')\nday_before_df = df[df['Date']==day_before].set_index('State/UT')\ntemp = pd.merge(left = latest_day_df, right = day_before_df, on='State/UT', suffixes=('_lat', '_bfr'), how='outer')\nlatest_day_df['New cases'] = temp['Total cases_lat'] - temp['Total cases_bfr']\nlatest = latest_day_df.reset_index()\nlatest.fillna(1, inplace=True)\n\n"

In [21]:
df_india["Lat"] = ""
df_india["Long"] = ""
for index in df_india.index :
    df_india.loc[df_india.index == index,"Lat"] = locations[index][0]
    df_india.loc[df_india.index == index,"Long"] = locations[index][1]

# **Overall view of the situation**

In [22]:
#option

ac= data['Active cases'].sum()
rvd = data['Cured/Discharged/Migrated'].sum()
dth = data['Death'].sum()
fig = go.Figure(data=[go.Pie(labels=['Active Cases','Cured','Death'],
                             values= [ac,rvd,dth],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#2fcc41','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Current Situation in India according to https://www.mohfw.gov.in',plot_bgcolor='rgb(275, 270, 273)')
fig.show()


In [23]:
#Overall 
ac= df_india['active'].sum()
rvd = df_india['recovered'].sum()
dth = df_india['deaths'].sum()
fig = go.Figure(data=[go.Pie(labels=['Active Cases','Cured','Death'],
                             values= [ac,rvd,dth],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#2fcc41','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Current Situation in India according to realtime data from https://www.covid19india.org',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [25]:
'''
pdata = pd.read_csv('../input/covid19-corona-virus-india-dataset/patients_data.csv')
pdata['p_id'] = pdata['patient_number'].apply(lambda x : 'P'+str(x))
pdata['nationality'] = pdata['nationality'].replace('Indian', 'India')
fig = px.pie(pdata['gender'].dropna(), 'gender', color_discrete_sequence = ['#85c451','#ca64cc'], 
             title='Male and Female')
fig.data[0].textinfo = 'label+text+value+percent'
fig.show('
'''

# **Trend of COVID-19 cases in India**

In [26]:
perday = df.groupby(['Date'])['Confirmed'].sum().reset_index().sort_values('Confirmed',ascending = True)
perday['New Daily Cases'] = perday['Confirmed'].sub(perday['Confirmed'].shift())
perday['New Daily Cases'].iloc[0] = perday['Confirmed'].iloc[0]
perday['New Daily Cases'] = perday['New Daily Cases'].astype(int)
# New COVID-19 cases reported daily in India
import plotly.express as px
fig = px.bar(perday, x="Date", y="New Daily Cases", barmode='group',height=500,color = "New Daily Cases",
            color_discrete_sequence = px.colors.sequential.Plasma_r)
fig.update_layout(title_text='New COVID-19 cases reported daily in India',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Confirmed'],
                    mode='lines+markers',marker_color='blue',name='Total Cases'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'],y=date_wise_data['Active'], 
                mode='lines+markers',marker_color='purple',name='Active'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Cured'],
                mode='lines+markers',marker_color='green',name='Recovered'))
fig.add_trace(go.Scatter(x=date_wise_data['Date'], y=date_wise_data['Deaths'], 
                mode='lines+markers',marker_color='red',name='Deaths'))
fig.update_layout(title_text='Trend of Weekly Coronavirus Cases in India',plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [28]:
cnf = '#263fa3' # confirmed - blue
act = '#fe9801' # active case - yellow
rec = '#21bf73' # recovered - green
dth = '#de260d' # death - red
tmp = date_wise_data.melt(id_vars="Date",value_vars=['Deaths','Cured' ,'Active','Confirmed'],
                 var_name='Case',value_name='Count')
fig = px.area(tmp, x="Date", y="Count",color='Case',
              title='Trend Over Weeks',color_discrete_sequence = [dth,rec,act,cnf])
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=700, height=600)
fig.show()

In [29]:
df_india_data = df[['Date', 'State/UT','Confirmed','Cured','Deaths','Active']]

In [30]:
spread = df_india_data.groupby(['Date', 'State/UT'])['Confirmed'].sum().reset_index().sort_values('Confirmed', ascending=True)
fig = px.area(spread, x="Date", y="Confirmed",color='State/UT',title='State Wise Spread over time',height=500,
              color_discrete_sequence = ex.colors.cyclical.Edge)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=700, height=600)

In [31]:
temp = date_wise_data.copy()
temp['Recovery Rate'] = temp['Cured']/temp['Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Recovery Rate'],
                    mode='lines+markers',marker_color='green'))
fig.update_layout(title_text = 'Cummulative Recovery Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [32]:
temp = date_wise_data.copy()
temp['Mortality Rate'] = temp['Deaths']/temp['Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Mortality Rate'],
                    mode='lines+markers',marker_color='red'))
fig.update_layout(title_text = 'Cummulative Mortality Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [33]:
agegroup = pd.read_csv('../input/covid19-in-india/AgeGroupDetails.csv')
fig = go.Figure()
fig.add_trace(go.Scatter(x=agegroup['AgeGroup'], y=agegroup['TotalCases'],line_shape='spline',
                        hovertext=["Percentage= 3.18%", "Percentage= 3.90%", "Percentage= 24.86%", "Percentage= 21.10%", "Percentage= 16.18%", "Percentage= 11.13%", "Percentage= 12.86%", "Percentage= 16.18%"],
                        hoverinfo='text', fill='tonexty',fillcolor = 'orange')) 
fig.update_layout(title="Age wise Trend",yaxis_title="Total cases",xaxis_title="Age Group")
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=800,height=600)
fig.show()

# **Covid-19 State Wise insight**

In [34]:
temp = data.sort_values('Total cases', ascending=True)
fig = go.Figure(data=[
    go.Bar(name='Active', y=temp['Name of State / UT'], x=temp['Active cases'], orientation='h'),
    go.Bar(name='Death', y=temp['Name of State / UT'], x=temp['Death'], orientation='h'),
    go.Bar(name='Cured', y=temp['Name of State / UT'], x=temp['Cured/Discharged/Migrated'], orientation='h')])
fig.update_layout(barmode='stack', height=900)
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(title_text='Active Cases,Cured,Deaths in Different States of India',
                  plot_bgcolor='rgb(275, 270, 273)',width=700, height=800)
fig.show()

In [35]:
#
'''
temp = pdata[['age_bracket', 'current_status']].dropna()
fig = px.histogram(temp, x='age_bracket', color='current_status', nbins=50, 
                   category_orders = {'current_status': ['Deceased', 'Recovered', 'Hospitalized', 'Migrated']},
                   color_discrete_sequence = ['#ff4d4d', '#21bf73', '#005691', '#3bb4c1'],
                  title='Distribution of ages of patients')
fig.show()
'''

"\ntemp = pdata[['age_bracket', 'current_status']].dropna()\nfig = px.histogram(temp, x='age_bracket', color='current_status', nbins=50, \n                   category_orders = {'current_status': ['Deceased', 'Recovered', 'Hospitalized', 'Migrated']},\n                   color_discrete_sequence = ['#ff4d4d', '#21bf73', '#005691', '#3bb4c1'],\n                  title='Distribution of ages of patients')\nfig.show()\n"

In [36]:
#old 
'''
temp = data.sort_values('Total cases', ascending=True)
fig = go.Figure(data=[
    go.Bar(name='Active', y=temp['State/UT'], x=temp['Active'], orientation='h'),
    go.Bar(name='Death', y=temp['State/UT'], x=temp['Death'], orientation='h'),
    go.Bar(name='Cured', y=temp['State/UT'], x=temp['Cured/Discharged/Migrated'], orientation='h')])
fig.update_layout(barmode='stack', height=900)
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(title_text='Active Cases,Cured,Deaths in Different States of India',
                  plot_bgcolor='rgb(275, 270, 273)',width=600, height=800)
fig.show()
'''

"\ntemp = data.sort_values('Total cases', ascending=True)\nfig = go.Figure(data=[\n    go.Bar(name='Active', y=temp['State/UT'], x=temp['Active'], orientation='h'),\n    go.Bar(name='Death', y=temp['State/UT'], x=temp['Death'], orientation='h'),\n    go.Bar(name='Cured', y=temp['State/UT'], x=temp['Cured/Discharged/Migrated'], orientation='h')])\nfig.update_layout(barmode='stack', height=900)\nfig.update_traces(textposition='outside')\nfig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')\nfig.update_layout(title_text='Active Cases,Cured,Deaths in Different States of India',\n                  plot_bgcolor='rgb(275, 270, 273)',width=600, height=800)\nfig.show()\n"

In [37]:
#colors 
'''
color_discrete_sequence = px.colors.sequential.Plasma_r, ex.colors.cyclical.IceFire, ex.colors.cyclical.Edge'
'''

"\ncolor_discrete_sequence = px.colors.sequential.Plasma_r, ex.colors.cyclical.IceFire, ex.colors.cyclical.Edge'\n"

In [38]:
temp = data.sort_values('Total cases', ascending=False)
state_order = temp['Name of State / UT']
fig = px.bar(temp,x="Total cases", y="Name of State / UT", color='Name of State / UT', title='State Wise Confirmed Cases', orientation='h', text='Total cases', 
             height=900,color_discrete_sequence = px.colors.sequential.Plasma_r)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [39]:
#original 
'''
fig = px.treemap(latest, path=['State/UT'], values="Cured" , height=500,
                 title='State Wise Number of Cured Cases', 
                 color_discrete_sequence = ex.colors.cyclical.IceFire)
fig.data[0].textinfo = 'label+text+value'
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=700, height=650)
fig.show()
'''

'\nfig = px.treemap(latest, path=[\'State/UT\'], values="Cured" , height=500,\n                 title=\'State Wise Number of Cured Cases\', \n                 color_discrete_sequence = ex.colors.cyclical.IceFire)\nfig.data[0].textinfo = \'label+text+value\'\nfig.update_layout(plot_bgcolor=\'rgb(275, 270, 273)\',width=700, height=650)\nfig.show()\n'

In [40]:
#new one
fig = px.treemap(data, path=["Name of State / UT"], values="Cured/Discharged/Migrated" , height=500,
                 title='State Wise Number of Cured Cases', 
                 color_discrete_sequence = ex.colors.cyclical.IceFire)
fig.data[0].textinfo = 'label+text+value'
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=700, height=650)
fig.show()

In [41]:
#old & not working
'''
fig = px.treemap(latest, path=['State/UT'], values="Deaths", height=500,
                 title='State Wise Number of Death', color_discrete_sequence = ex.colors.cyclical.Twilight)
fig.data[0].textinfo = 'label+text+value'
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=700, height=650)
fig.show()
'''

'\nfig = px.treemap(latest, path=[\'State/UT\'], values="Deaths", height=500,\n                 title=\'State Wise Number of Death\', color_discrete_sequence = ex.colors.cyclical.Twilight)\nfig.data[0].textinfo = \'label+text+value\'\nfig.update_layout(plot_bgcolor=\'rgb(275, 270, 273)\',width=700, height=650)\nfig.show()\n'

In [42]:
#updated
fig = px.treemap(data, path=["Name of State / UT"], values="Death", height=500,
                 title='State Wise Number of Death', color_discrete_sequence = ex.colors.cyclical.Twilight)
fig.data[0].textinfo = 'label+text+value'
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=700, height=650)
fig.show()

# Over the time Demographics Visualisation

In [43]:
india = folium.Map(location=[23,80], zoom_start=4,max_zoom=6,min_zoom=4,height=500,width="100%",tiles='cartodbpositron')
for i in range(0,len(df_india[df_india['confirmed']>0].index)):
    folium.Circle(
        location=[df_india.iloc[i]['Lat'], df_india.iloc[i]['Long']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+df_india.iloc[i].name+"</h5>"+
                    "<hr style='margin:10px;'>"+
                    "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        "<li>Confirmed: "+str(df_india.iloc[i]['confirmed'])+"</li>"+
        "<li>Active:   "+str(df_india.iloc[i]['active'])+"</li>"+
        "<li>Recovered:   "+str(df_india.iloc[i]['recovered'])+"</li>"+
        "<li>Deaths:   "+str(df_india.iloc[i]['deaths'])+"</li>"+
        
        "<li>Mortality Rate:   "+str(np.round(df_india.iloc[i]['deaths']/(df_india.iloc[i]['confirmed']+1)*100,2))+"</li>"+
        "</ul>"
        ,
        radius=(int(np.log2(df_india.iloc[i]['confirmed']+1)))*15000,
        color='red',
        fill_color='green',
        fill=True).add_to(india)
india

**COVID-19 Cases thtough time in different parts of india**

In [44]:
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)
tmp = df.copy()
tmp['Date'] = tmp['Date'].dt.strftime('%Y/%m/%d')
fig = px.scatter_geo(tmp, lat="Latitude", lon="Longitude", color='Confirmed', size='Confirmed', projection="natural earth",
                     hover_name="State/UT", scope='asia', animation_frame="Date",
                     color_continuous_scale=px.colors.diverging.curl,center={'lat':20, 'lon':78}, 
                     range_color=[0, max(tmp['Confirmed'])])
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [45]:
#old

'''
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)
tmp = df.copy()
tmp['Date'] = tmp['Date'].dt.strftime('%Y/%m/%d')
fig = px.scatter_geo(tmp, lat="Latitude", lon="Longitude", color='Total cases', size='Total cases', projection="natural earth",
                     hover_name="State/UT", scope='asia', animation_frame="Date",
                     color_continuous_scale=px.colors.diverging.curl,center={'lat':20, 'lon':78}, 
                     range_color=[0, max(tmp['Total cases'])])
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.show()
'''

'\nfrom plotly.offline import init_notebook_mode\nimport plotly.graph_objs as go\ninit_notebook_mode(connected=True)\ntmp = df.copy()\ntmp[\'Date\'] = tmp[\'Date\'].dt.strftime(\'%Y/%m/%d\')\nfig = px.scatter_geo(tmp, lat="Latitude", lon="Longitude", color=\'Total cases\', size=\'Total cases\', projection="natural earth",\n                     hover_name="State/UT", scope=\'asia\', animation_frame="Date",\n                     color_continuous_scale=px.colors.diverging.curl,center={\'lat\':20, \'lon\':78}, \n                     range_color=[0, max(tmp[\'Total cases\'])])\nfig.update_layout(plot_bgcolor=\'rgb(275, 270, 273)\')\nfig.show()\n'

In [46]:
covid19 = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')
covid19_recovered = pd.read_csv('../input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv')
covid19_deaths = pd.read_csv('../input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv')
covid19_confirmed = pd.read_csv('../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')
covid19.drop(columns='SNo')
covid19[["Confirmed","Deaths","Recovered"]] = covid19[["Confirmed","Deaths","Recovered"]].astype(int) 
covid19['Active'] = covid19['Confirmed'] - covid19['Deaths'] - covid19['Recovered']
covid19.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1,0,0,1
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14,0,0,14
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6,0,0,6
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1,0,0,1
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0,0,0,0


In [47]:
world = covid19.groupby('ObservationDate')[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index()
world.sort_values(by ='ObservationDate',ascending=0)
world[['Confirmed', 'Deaths','Recovered','Active']] = world[['Confirmed', 'Deaths','Recovered','Active']].astype(int)
world = world.sort_values(by = 'ObservationDate', ascending = 0)
world.head()

,ObservationDate,Confirmed,Deaths,Recovered,Active
68,03/30/2020,782365,37582,164566,580217
67,03/29/2020,720117,33925,149082,537110
66,03/28/2020,660706,30652,139415,490639
65,03/27/2020,593291,27198,130915,435178
64,03/26/2020,529591,23970,122150,383471


In [48]:
df19 = covid19[covid19['ObservationDate'] == max(covid19['ObservationDate'])]
country = df19.groupby(["Country/Region"])[['Confirmed','Deaths','Recovered','Active']].sum().reset_index().sort_values("Confirmed",ascending=0).reset_index(drop=True)
country.head()

,Country/Region,Confirmed,Deaths,Recovered,Active
0,US,161807,2978,5644,153185
1,Italy,101739,11591,14620,75528
2,Spain,87956,7716,16780,63460
3,Mainland China,81478,3304,75790,2384
4,Germany,66885,645,13500,52740


In [49]:
confirmed = covid19.groupby('ObservationDate').sum()['Confirmed'].reset_index()
deaths = covid19.groupby('ObservationDate').sum()['Deaths'].reset_index()
recovered = covid19.groupby('ObservationDate').sum()['Recovered'].reset_index()
active = covid19.groupby('ObservationDate').sum()['Active'].reset_index()

In [50]:
china = covid19[covid19['Country/Region'] == 'Mainland China'].reset_index()
china_obs_date = china.groupby('ObservationDate')[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index()
china_obs_date.sort_values(by = 'ObservationDate', ascending = 0).head()

,ObservationDate,Confirmed,Deaths,Recovered,Active
68,03/30/2020,81478,3304,75790,2384
67,03/29/2020,81444,3300,75460,2684
66,03/28/2020,81401,3295,74978,3128
65,03/27/2020,81345,3292,74600,3453
64,03/26/2020,81298,3287,74061,3950


In [51]:
Italy = covid19[covid19['Country/Region'] == 'Italy'].reset_index()
Italy_obs_date = Italy.groupby('ObservationDate')[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index()
Italy_obs_date.sort_values(by = 'ObservationDate', ascending = 0).head()

,ObservationDate,Confirmed,Deaths,Recovered,Active
59,03/30/2020,101739,11591,14620,75528
58,03/29/2020,97689,10779,13030,73880
57,03/28/2020,92472,10023,12384,70065
56,03/27/2020,86498,9134,10950,66414
55,03/26/2020,80589,8215,10361,62013


In [52]:
US = covid19[covid19['Country/Region'] == 'US'].reset_index()
US_obs_date = US.groupby('ObservationDate')[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index()
US_obs_date.sort_values(by = 'ObservationDate', ascending = 0).head()

,ObservationDate,Confirmed,Deaths,Recovered,Active
68,03/30/2020,161807,2978,5644,153185
67,03/29/2020,140886,2467,2665,135754
66,03/28/2020,121478,2026,1072,118380
65,03/27/2020,101657,1581,869,99207
64,03/26/2020,83836,1209,681,81946


In [53]:
India = covid19[covid19['Country/Region'] == 'India'].reset_index()
India_obs_date = India.groupby('ObservationDate')[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index()
India_obs_date.sort_values(by = 'ObservationDate', ascending = 0).head()

,ObservationDate,Confirmed,Deaths,Recovered,Active
60,03/30/2020,1251,32,102,1117
59,03/29/2020,1024,27,95,902
58,03/28/2020,987,24,84,879
57,03/27/2020,887,20,73,794
56,03/26/2020,727,20,45,662


In [54]:
Spain = covid19[covid19['Country/Region'] == 'Spain'].reset_index()
Spain_obs_date = Spain.groupby('ObservationDate')[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index()
Spain_obs_date.sort_values(by = 'ObservationDate', ascending = 0).head()

,ObservationDate,Confirmed,Deaths,Recovered,Active
58,03/30/2020,87956,7716,16780,63460
57,03/29/2020,80110,6803,14709,58598
56,03/28/2020,73235,5982,12285,54968
55,03/27/2020,65719,5138,9357,51224
54,03/26/2020,57786,4365,7015,46406


In [55]:
Germany = covid19[covid19['Country/Region'] == 'Germany'].reset_index()
Germany_obs_date = Germany.groupby('ObservationDate')[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index()
Germany_obs_date.sort_values(by = 'ObservationDate', ascending = 0).head()

,ObservationDate,Confirmed,Deaths,Recovered,Active
62,03/30/2020,66885,645,13500,52740
61,03/29/2020,62095,533,9211,52351
60,03/28/2020,57695,433,8481,48781
59,03/27/2020,50871,342,6658,43871
58,03/26/2020,43938,267,5673,37998


In [56]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = china_obs_date['ObservationDate'],
                         y = china_obs_date['Confirmed'],
                         mode='lines+markers',
                         name='China',
                         marker_color='red'
                         ))
fig.add_trace(go.Scatter(x = Italy_obs_date['ObservationDate'],
                         y = Italy_obs_date['Confirmed'],
                         mode='lines+markers',
                         name='Italy',
                         marker_color='green'
                         ))
fig.add_trace(go.Scatter(x = US_obs_date['ObservationDate'],
                         y = US_obs_date['Confirmed'],
                         mode='lines+markers',
                         name='US',
                         marker_color='blue'
                         ))
fig.add_trace(go.Scatter(x = India_obs_date['ObservationDate'],
                         y = India_obs_date['Confirmed'],
                         mode='lines+markers',
                         name='India',
                         marker_color='orange'
                         ))
fig.add_trace(go.Scatter(x = Spain_obs_date['ObservationDate'],
                         y = Spain_obs_date['Confirmed'],
                         mode='lines+markers',
                         name='Spain',
                         marker_color='#F1BF00'
                         ))
fig.add_trace(go.Scatter(x = Germany_obs_date['ObservationDate'],
                         y = Germany_obs_date['Confirmed'],
                         mode='lines+markers',
                         name='Germany',
                         marker_color='black'
                         ))
fig.update_layout(title='Trend of Confirmed Cases in India & other Top-5 Countries',
                  xaxis = dict(title = 'Dates'),
                  yaxis = dict(title = 'Cases'))
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=700)
fig.show()

In [57]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = china_obs_date['ObservationDate'],
                         y = china_obs_date['Recovered'],
                         mode='lines+markers',
                         name='China',
                         marker_color='red'
                         ))
fig.add_trace(go.Scatter(x = Italy_obs_date['ObservationDate'],
                         y = Italy_obs_date['Recovered'],
                         mode='lines+markers',
                         name='Italy',
                         marker_color='green'
                         ))
fig.add_trace(go.Scatter(x = US_obs_date['ObservationDate'],
                         y = US_obs_date['Recovered'],
                         mode='lines+markers',
                         name='US',
                         marker_color='blue'
                         ))
fig.add_trace(go.Scatter(x = India_obs_date['ObservationDate'],
                         y = India_obs_date['Recovered'],
                         mode='lines+markers',
                         name='India',
                         marker_color='orange'
                         ))
fig.add_trace(go.Scatter(x = Spain_obs_date['ObservationDate'],
                         y = Spain_obs_date['Recovered'],
                         mode='lines+markers',
                         name='Spain',
                         marker_color='#F1BF00'
                         ))
fig.add_trace(go.Scatter(x = Germany_obs_date['ObservationDate'],
                         y = Germany_obs_date['Recovered'],
                         mode='lines+markers',
                         name='Germany',
                         marker_color='black'
                         ))
fig.update_layout(title='Trend of Cured cases in India & other Top-5 Countries',
                  xaxis = dict(title = 'Observation Dates'),
                  yaxis = dict(title = 'Number of Cases'))
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=700)
fig.show()


In [58]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = china_obs_date['ObservationDate'],
                         y = china_obs_date['Deaths'],
                         mode='lines+markers',
                         name='China',
                         marker_color='red'))
fig.add_trace(go.Scatter(x = Italy_obs_date['ObservationDate'],
                         y = Italy_obs_date['Deaths'],
                         mode='lines+markers',
                         name='Italy',
                         marker_color='green'))
fig.add_trace(go.Scatter(x = US_obs_date['ObservationDate'],
                         y = US_obs_date['Deaths'],
                         mode='lines+markers',
                         name='US',
                         marker_color='blue'))
fig.add_trace(go.Scatter(x = India_obs_date['ObservationDate'],
                         y = India_obs_date['Deaths'],
                         mode='lines+markers',
                         name='India',
                         marker_color='orange'))
fig.add_trace(go.Scatter(x = Spain_obs_date['ObservationDate'],
                         y = Spain_obs_date['Deaths'],
                         mode='lines+markers',
                         name='Spain',
                         marker_color='#F1BF00'))
fig.add_trace(go.Scatter(x = Germany_obs_date['ObservationDate'],
                         y = Germany_obs_date['Deaths'],
                         mode='lines+markers',
                         name='Germany',
                         marker_color='black'))
fig.update_layout(title='Trend of Deaths in India & Top-5 Countries',
                  xaxis = dict(title = 'Observation Dates'),
                  yaxis = dict(title = 'Number of Cases'))
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=700)
fig.show()

In [59]:
fig = go.Figure()
fig.add_trace(go.Bar(x = country['Country/Region'][0:5],
                     y = country['Confirmed'][0:5],
                     marker_color=country['Confirmed'][0:5],
                     text = country['Confirmed'][0:5],
                     textposition = 'outside',
                     name = 'Confirmed'))
fig.add_trace(go.Bar(x = country['Country/Region'][country[country['Country/Region'] == 'India'].index],
                     y = country['Confirmed'][country[country['Country/Region'] == 'India'].index],
                     marker_color='white',
                     text = country['Confirmed'][country[country['Country/Region'] == 'India'].index],
                     textposition = 'outside',
                     showlegend=False))
fig.update_layout(title='Confirmed cases - Top 5 Countries vs India',
                  xaxis_title="Countries",
                  yaxis_title="Confirmed Cases",)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [60]:
country_active = df19.groupby(["Country/Region"])[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index().sort_values('Active',ascending=False).reset_index(drop=True)
fig = go.Figure()
fig.add_trace(go.Bar(x = country_active['Country/Region'][0:5],
                     y = country_active['Active'][0:5],
                     marker_color=country_active['Active'][0:5],
                     text = country_active['Active'][0:5],
                     textposition = 'outside',
                     name = 'Active Cases'))
fig.add_trace(go.Bar(x = country_active['Country/Region'][country_active[country_active['Country/Region'] == 'India'].index],
                     y = country_active['Active'][country_active[country_active['Country/Region'] == 'India'].index],
                     marker_color='orange',
                     text = country_active['Active'][country_active[country_active['Country/Region'] == 'India'].index],
                     textposition = 'outside',
                     showlegend=False))
fig.update_layout(title='Active cases - Top 5 Countries vs India',
                  xaxis_title="Countries",
                  yaxis_title="Active Cases")
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)')
fig.update_layout(barmode='stack',width=600, height=600)
fig.update_traces(textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [61]:
country_death = df19.groupby(["Country/Region"])[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index().sort_values("Deaths",ascending=False).reset_index(drop=True)

fig = go.Figure()

fig.add_trace(go.Bar(x = country_death['Country/Region'][0:5],
                     y = country_death['Deaths'][0:5],
                     marker_color=country_death['Deaths'][0:5],
                     text = country_death['Deaths'][0:5],
                     textposition = 'outside',
                     name = 'Deaths',))

fig.add_trace(go.Bar(x = country_death['Country/Region'][country_death[country_death['Country/Region'] == 'India'].index],
                     y = country_death['Deaths'][country_death[country_death['Country/Region'] == 'India'].index],
                     marker_color='red',
                     text = country_death['Deaths'][country_death[country_death['Country/Region'] == 'India'].index],
                     textposition = 'outside',
                     showlegend=False,))

fig.update_layout(title='Death cases - Top 5 Countries vs India',
                  xaxis_title="Countries",
                  yaxis_title="Deaths")
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [62]:
country_recovered = df19.groupby(["Country/Region"])[['Confirmed', 'Deaths','Recovered','Active']].sum().reset_index().sort_values('Recovered',ascending=False).reset_index(drop=True)
fig = go.Figure()
fig.add_trace(go.Bar(x = country_recovered['Country/Region'][0:5],
                     y = country_recovered['Recovered'][0:5],
                     marker_color=country_recovered['Recovered'][0:5],
                     text = country_recovered['Recovered'][0:5],
                     textposition = 'outside',
                     name = 'Recovered'))
fig.add_trace(go.Bar(x = country_recovered['Country/Region'][country_recovered[country_recovered['Country/Region'] == 'India'].index],
                     y = country_recovered['Recovered'][country_recovered[country_recovered['Country/Region'] == 'India'].index],
                     marker_color='green',
                     text = country_recovered['Recovered'][country_recovered[country_recovered['Country/Region'] == 'India'].index],
                     textposition = 'outside',
                     showlegend=False))
fig.update_layout(title='Recovered cases - Top 5 Countries vs India',
                  xaxis_title="Countries",
                  yaxis_title="Recovered Cases")
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

**Note: This work is highly inspired from few other kaggle kernels , github sources and other data science resources. Any traces of replications, which may appear , is purely co-incidental. Due respect & credit to all my fellow kagglers.**